In [54]:
import networkx as nx
import matplotlib.pyplot as plt

# Построим граф - кварто-квинтовый круг, по которому можно двигаться только по одному шагу
# При таком движении смена тональностей происходит наиболее незаметно (менятеся только один знак при ключе)
# Определяем вершины (тональности) и рёбра (переходы на кварту, квинту, в параллельные миноры)
notes_maj = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
notes_min = ["Cm", "C#m", "Dm", "D#m", "Em", "Fm", "F#m", "Gm", "G#m", "Am", "A#m", "Bm"]
keys = notes_maj + notes_min

# Создаём словари для четвёртых, пятых ступеней и параллельных тональностей
fourth = {keys[i]: keys[(i + 5) % 12] if i < 12 else keys[(i + 5) % 12 + 12] for i in range(24)}
fifth = {keys[i]: keys[(i + 7) % 12] if i < 12 else keys[(i + 7) % 12 + 12] for i in range(24)}
parallel = {keys[i]: keys[(i + 9) % 12 + 12] if i < 12 else keys[(i - 9) % 12] for i in range(24)}

# Создаём граф и добавляем узлы
G = nx.DiGraph()
G.add_nodes_from(keys)

# Создаём графовые рёбра для каждой тональности
for i in range(24):
    G.add_edge(keys[i], keys[i], quality='root'),
    G.add_edge(keys[i], fourth[keys[i]], quality='fourth'),
    G.add_edge(keys[i], fifth[keys[i]], quality='fifth'),
    G.add_edge(keys[i], parallel[fifth[keys[i]]], quality='fifth_par'),
    G.add_edge(keys[i], parallel[fourth[keys[i]]], quality='fourth_par'),
    G.add_edge(keys[i], parallel[keys[i]], quality='par')

In [57]:
import random


# Указываем текущую тональность (компоненту связности), и рандомим следующую тональность
def in_key(cur_key):
    if cur_key not in keys:
        print('ERROR: There`s no key named this way')
    else:
        in_key = {v['quality']: k for k, v in G[cur_key].items()}

    return in_key

def change_key(cur_key):
    in_key_chords = in_key(cur_key)
    chng_key = random.choice(list(in_key_chords.values())[1:])
    return chng_key

'G'

In [ ]:
!pip3 install pretty_midi
!pip3 install mido

In [66]:
import mido
import pretty_midi
from music21 import chord, pitch

def chords_format(listOfChords):
    chords = []
    for chord_obj in listOfChords:
        chord_symbol = chord_obj.pitchedCommonName  # Пример: 'C major triad', 'A minor triad'

        # Преобразуем в более простой формат
        if 'major' in chord_symbol:
            chord_name = chord_obj.root().name + ""
        elif 'minor' in chord_symbol:
            chord_name = chord_obj.root().name + "m"

        chords.append(chord_name)

    return chords

def midi_to_chords(midi_file_path):
    # Загрузка MIDI-файла
    midi_data = pretty_midi.PrettyMIDI(midi_file_path)

    # Инициализация списка для аккордов
    chords = []

    # Перебираем все инструменты в MIDI-файле (обычно мелодии на инструменте Piano)
    for instrument in midi_data.instruments:
        if not instrument.is_drum:  # Пропускаем ударные
            # Группируем ноты в аккорды по времени начала
            notes_by_start = {}
            for note in instrument.notes:
                start_time = round(note.start, 2)  # Округляем время до 2 знаков для удобства
                if start_time not in notes_by_start:
                    notes_by_start[start_time] = []
                notes_by_start[start_time].append(note)

            # Определяем аккорд по набору нот
            for start_time, notes in notes_by_start.items():
                # Извлекаем высоты нот
                pitches = [note.pitch for note in notes]

                # Преобразуем в объекты pitch для анализа аккорда
                chord_notes = [pitch.Pitch(midi) for midi in pitches]

                # Анализируем аккорд и добавляем в список
                chord_symbol = chord.Chord(chord_notes)
                chords.append(chord_symbol)

    return chords

midi_file_path = 'fifth.mid'
chord_sequence = midi_to_chords(midi_file_path)
chords_formated = chords_format(chord_sequence)
print(chords_formated)

['Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'G', 'Bm', 'D', 'Em', 'G', 'D', 'A', 'G']


In [83]:
# Определим тональность по списку аккордов
# ВНИМАНИЕ: метод очень наивный, но используемый для простых композиций
from collections import Counter


def detect_key(chords):
    key_matches = Counter()

    for key in list(G.nodes())[:12]:
        matches = sum(1 for chord in chords if chord in list(dict(G[key]).keys()))
        key_matches[key] += matches

    # Находим тональность с наибольшим количеством совпадений
    most_likely_key = key_matches.most_common(1)[0]
    return most_likely_key

detect_key(chords_formated)

('G', 13)